In [25]:
import pandas as pd
print(pd.__version__)
import numpy as np

# UTILITARIES
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import tensorflow as tf
import os

# Check if GPU is available and TensorFlow is using it
if tf.test.is_gpu_available():
    print('TensorFlow is using GPU')
else:
    print('TensorFlow is not using GPU')

1.5.3
TensorFlow is not using GPU


In [26]:
env_="local"
PATH = "/kaggle/input/"
PATH="../data/"
if env_=="COLAB_INIT":
  from google.colab import drive
  drive.mount('/content/drive')
  ! pip install kaggle
  ! mkdir ~/.kaggle
  os.getcwd()
  ! pip install Bio


MAIN_DIR = f"{PATH}cafa-5-protein-function-prediction"

class config:
    train_sequences_path = MAIN_DIR  + "/Train/train_sequences.fasta"
    train_labels_path = MAIN_DIR + "/Train/train_terms.tsv"
    test_sequences_path = MAIN_DIR + "/Test (Targets)/testsuperset.fasta"
    
    num_labels = 1500
    n_epochs = 5
    batch_size = 128
    lr = 0.001

    env = env_

    
TRAIN = False
embeddings_source= "T5"

PATH_DATAFRAMES=PATH_LABELS="../data/data_train"
PATH_DATAFRAMES_2D="data/data_train_embeds_2d"
if env_=="kaggle":
    PATH_DATAFRAMES="/kaggle/input/dataframes-train-cafa/data_train"
    PATH_DATAFRAMES_2D="/kaggle/input/dataframes-train-cafa/data_train_embeds_2d"
    PATH_LABELS="/kaggle/input/dataframes-train-cafa/"

In [27]:
import pandas as pd

sub = pd.read_csv(f"{PATH}cafa-5-protein-function-prediction/sample_submission.tsv", sep= "\t", header = None)
sub.columns = ["The Protein ID", "The Gene Ontology term (GO) ID", "Predicted link probability that GO appear in Protein"]
sub.head(5)

,The Protein ID,The Gene Ontology term (GO) ID,Predicted link probability that GO appear in Protein
0,A0A0A0MRZ7,GO:0000001,0.123
1,A0A0A0MRZ7,GO:0000002,0.123
2,A0A0A0MRZ8,GO:0000001,0.123
3,A0A0A0MRZ8,GO:0000002,0.123
4,A0A0A0MRZ9,GO:0000001,0.123


In [28]:

PATHS = {
    "PATH": PATH, 
    "PATH_LABELS": PATH_LABELS,
    "PATH_DATAFRAMES_2D": PATH_DATAFRAMES_2D,
    "CHECKPOINT": {"local":'../results/models_folds_aspect/models_{}_{}', "kaggle": '/kaggle/working/models_{}_{}'},
    "PREDICTIONS": {"local":'../results/preds_cafa_embeddings_bp/preds_fold{}.tsv', "kaggle": 'preds_fold{}.tsv'},
    "LABELS": {"local":'../results/preds_cafa_embeddings_bp_aspect/labels_used_fold{}.tsv', "kaggle": 'labels_used_fold{}.tsv'},
}

In [29]:
from Bio import SeqIO
print("Loading train set ProtBERT Embeddings...")
fasta_train = SeqIO.parse(config.train_sequences_path, "fasta")
print("Total Nb of Elements : ", len(list(fasta_train)))
fasta_test= SeqIO.parse(config.test_sequences_path, "fasta")
print("Total Nb of Elements : ", len(list(fasta_test)))

Loading train set ProtBERT Embeddings...
Total Nb of Elements :  142246
Total Nb of Elements :  141865


In [30]:
weights=pd.read_csv(f"{PATH}cafa-5-protein-function-prediction/IA.txt",sep="\t",header=None,index_col=0).rename(columns={1:"weight"})


In [31]:
 labels = pd.read_csv(config.train_labels_path, sep = "\t")
 labels.shape

(5363863, 3)

In [32]:
 labels= labels.loc[~labels.term.isin(list(weights[weights.weight==0].index))]
 labels.shape

(4861356, 3)

In [33]:
# Directories for the different embedding vectors : 
embeds_map = {
    "T5" : "my-t5embeds",
    "ProtBERT" : "protbert-embeddings-for-cafa5",
    "EMS2" : "cafa-5-ems-2-embeddings-numpy"
}

# Length of the different embedding vectors :
embeds_dim = {
    "T5" : 1024,
    "ProtBERT" : 1024,
    "EMS2" : 1280
}

In [34]:
 LEARNING_RATE=0.001
 WARMUP_STEPS=7
 class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, warmup_steps=1):
        super(CustomSchedule, self).__init__()

        self.initial_lr = tf.cast(initial_lr, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        return tf.math.minimum(self.initial_lr, self.initial_lr * (step/self.warmup_steps))  
    
    def get_config(self):
        return {"initial_learning_rate": self.initial_lr}
    

'''
PredictionFnCallback is used for:
1. Loading validation data
2. FOGModel data preparation
3. Prediction
4. Scoring and save

'''

'\nPredictionFnCallback is used for:\n1. Loading validation data\n2. FOGModel data preparation\n3. Prediction\n4. Scoring and save\n\n'

In [35]:
import tensorflow as tf

#models_dp_v0
class MultiLayerPerceptron_v0(tf.keras.Model):

    def __init__(self, num_classes):
        super(MultiLayerPerceptron, self).__init__()
                
         
        self.linear1 = tf.keras.layers.Dense(256)
        #self.activation1 = tf.keras.layers.ReLU()
        self.linear2 = tf.keras.layers.Dense(128)
        #self.activation2 = tf.keras.layers.ReLU()
        self.linear2 = tf.keras.layers.Dense(56)
        
        self.linear3 = tf.keras.layers.Dense(num_classes,activation="sigmoid")

    def call(self, x):

        x = self.linear1(x)
        #x = self.activation1(x)
        x = self.linear2(x)
        # x = self.activation2(x)
        x = self.linear3(x)
        return x
import tensorflow as tf



#models_dp_v1
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, units):
        super(ResidualBlock, self).__init__()
        self.fc1 = tf.keras.layers.Dense(units, activation='relu')
        self.fc2 = tf.keras.layers.Dense(units, activation='relu')

    def call(self, x):
        residual = x
        x = self.fc1(x)
        x = self.fc2(x)
        x = x + residual
        return x

class MultiLayerPerceptron(tf.keras.Model):
    def __init__(self, num_classes):
        super(MultiLayerPerceptron, self).__init__()
        self.fc1 = tf.keras.layers.Dense(128, activation='relu')
        self.fc2 = tf.keras.layers.Dense(56, activation='relu')
        self.res_block1 = ResidualBlock(56)
        self.res_block2 = ResidualBlock(56)
        self.fc3 = tf.keras.layers.Dense(num_classes, activation='sigmoid')

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.fc3(x)
        return x


#models_dp_v2
class ResidualBlock_v2(tf.keras.layers.Layer):
    def __init__(self, units):
        super(ResidualBlock, self).__init__()
        self.fc1 = tf.keras.layers.Dense(units, activation='relu')
        self.fc2 = tf.keras.layers.Dense(units, activation='relu')

    def call(self, x):
        residual = x
        x = self.fc1(x)
        x = self.fc2(x)
        x = x + residual
        return x

class MultiLayerPerceptron_v2(tf.keras.Model):
    def __init__(self, num_classes):
        super(MultiLayerPerceptron, self).__init__()
        self.res_block1 = ResidualBlock(1024)
        self.fc1 = tf.keras.layers.Dense(128, activation='relu')
        self.res_block2 = ResidualBlock(512)
        self.fc2 = tf.keras.layers.Dense(256, activation='relu')
        self.res_block3 = ResidualBlock(256)
        self.fc3 = tf.keras.layers.Dense(128, activation='relu')
        self.fc4 = tf.keras.layers.Dense(num_classes, activation='sigmoid')

    def call(self, x):
        # x = self.res_block1(x)
        x = self.fc1(x)
        # x = self.res_block2(x)
        # x = self.fc2(x)
        # x = self.res_block3(x)
        # x = self.fc3(x)
        x = self.fc4(x)
        return x




#models_cnn_v0
class CNN1D(tf.keras.Model):  #v0
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()
        # (batch_size, channels, embed_size)
        self.conv1 = tf.keras.layers.Conv1D(filters=7, kernel_size=3, dilation_rate=1, padding='same', activation='relu')
        # (batch_size, 3, embed_size)
        self.pool1 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)
        # (batch_size, 3, embed_size/2 = 512)
        self.conv2 = tf.keras.layers.Conv1D(filters=15, kernel_size=3, dilation_rate=1, padding='same', activation='relu')
        # (batch_size, 8, embed_size/2 = 512)
        self.pool2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)
        
        self.flatten=tf.keras.layers.Flatten()
        # (batch_size, 8, embed_size/4 = 256)
        self.fc1 = tf.keras.layers.Dense(units=128, activation='relu')
        self.fc2 = tf.keras.layers.Dense(units=num_classes,activation="sigmoid")

    def call(self, x):
        x = tf.reshape(x, [-1, tf.shape(x)[1],1])
        x=self.conv1(x)
        x = self.pool1(x)
        x = self.pool2(self.conv2(x))
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


#models_cnn_v0
class CNN1D(tf.keras.Model):  #v0
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()
        # (batch_size, channels, embed_size)
        self.conv1 = tf.keras.layers.Conv1D(filters=7, kernel_size=3, dilation_rate=1, padding='same', activation='relu')
        # (batch_size, 3, embed_size)
        self.pool1 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)
        # (batch_size, 3, embed_size/2 = 512)
        self.conv2 = tf.keras.layers.Conv1D(filters=15, kernel_size=3, dilation_rate=1, padding='same', activation='relu')
        # (batch_size, 8, embed_size/2 = 512)
        self.pool2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)
        
        self.flatten=tf.keras.layers.Flatten()
        # (batch_size, 8, embed_size/4 = 256)
        self.fc1 = tf.keras.layers.Dense(units=128, activation='relu')
        self.fc2 = tf.keras.layers.Dense(units=num_classes,activation="sigmoid")

    def call(self, x):
        x = tf.reshape(x, [-1, tf.shape(x)[1],1])
        x=self.conv1(x)
        x = self.pool1(x)
        x = self.pool2(self.conv2(x))
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x





# 7. Train the Model

In [36]:
import pandas as pd
if TRAIN:
    #X_train = np.load(os.path.join(PATH_DATAFRAMES,f"embeds_train_{embeddings_source}.npy"))
    #X_validate = np.load(os.path.join(PATH_DATAFRAMES,f"embeds_test_{embeddings_source}.npy"))
    #y_train = pd.read_pickle(os.path.join(PATH_LABELS,f"labels_train_{   config. num_labels}.pkl"))
    #y_validate = pd.read_pickle(os.path.join(PATH_LABELS,f"labels_test_{   config. num_labels}.pkl"))
    X=np.load(os.path.join(PATH_DATAFRAMES,f"embeds_{embeddings_source}.npy")).astype(np. float32)
    y=pd.read_pickle(os.path.join(PATH_LABELS,f"labels_{   config. num_labels}.pkl")).astype(np. float32)

    num_classes=y.shape[1]
    print(f" Num classes {num_classes}")
    print(X.shape,y.shape)
    w_use=weights.loc[y.columns].values
    print(w_use.shape)
    labels = pd.read_csv(config.train_labels_path, sep = "\t")
    terms=labels[["term","aspect"]].drop_duplicates().set_index("term").loc[y.columns]
    aspects={}
    for aspect,df_terms in terms.groupby("aspect"):
        terms_index=list(df_terms.index.values)
        display(df_terms.head(5))
        indices = list(np.where(np.isin(y.columns,    terms_index))[0])
        aspects[aspect]=indices
    print(terms.shape)


In [37]:
import tensorflow as tf

from sklearn.model_selection import KFold
n_splits = 5

# Crear una instancia de KFold
kfold = KFold(n_splits=n_splits)



In [38]:
from sklearn.metrics import f1_score

def compute_metrics_(tau_arr, g, pred, toi, n_gt, wn_gt=None, ic_arr=None):

    metrics = np.zeros((len(tau_arr), 7), dtype='float')  # cov, pr, rc, wpr, wrc, ru, mi

    for i, tau in enumerate(tau_arr):

        p = solidify_prediction(pred[:, toi], tau)

        # number of proteins with at least one term predicted with score >= tau
        metrics[i, 0] = (p.sum(axis=1) > 0).sum()

        # Terms subsets
        intersection = np.logical_and(p, g)  # TP

        

        # Subsets size
        n_pred = p.sum(axis=1)
        n_intersection = intersection.sum(axis=1)

        # Precision, recall
        metrics[i, 1] = np.divide(n_intersection, n_pred, out=np.zeros_like(n_intersection, dtype='float'),
                                  where=n_pred > 0).sum() 
        metrics[i, 2] = np.divide(n_intersection, n_gt, out=np.zeros_like(n_gt, dtype='float'), where=n_gt > 0).sum()

        if ic_arr is not None:
            # Terms subsets
            remaining = np.logical_and(np.logical_not(p), g)  # FN --> not predicted but in the ground truth
            mis = np.logical_and(p, np.logical_not(g))  # FP --> predicted but not in the ground truth

            # Weighted precision, recall
            #wn_pred = (p * ic_arr[toi]).sum(axis=1)
            wn_pred=np.dot(p , ic_arr[toi])
            #wn_intersection = (intersection * ic_arr[toi]).sum(axis=1)
            wn_intersection = np.dot(intersection ,ic_arr[toi])
            
            #print(f"Shape {wn_intersection.shape},{wn_pred.shape},  {n_intersection.shape},{n_pred.shape}")
            metrics[i, 3] = np.divide(wn_intersection.reshape(-1), wn_pred.reshape(-1), out=np.zeros_like(n_intersection, dtype='float'),
                                      where=wn_pred.reshape(-1) > 0).sum() 
            metrics[i, 4] = np.divide(wn_intersection.reshape(-1), wn_gt.reshape(-1), out=np.zeros_like(n_intersection, dtype='float'),
                                      where=wn_gt.reshape(-1) > 0).sum() 

            # Misinformation, remaining uncertainty
            #metrics[i, 5] = (remaining * ic_arr[toi]).sum(axis=1).sum()
            metrics[i, 5] = np.dot(remaining , ic_arr[toi]).sum()
            #metrics[i, 6] = (mis * ic_arr[toi]).sum(axis=1).sum()
            metrics[i, 6] = np.dot(mis , ic_arr[toi]).sum()
    return metrics
# computes the f metric for each precision and recall in the input arrays
def compute_f(pr, rc):
    n = 2 * pr * rc
    d = pr + rc
    return np.divide(n, d, out=np.zeros_like(n, dtype=float), where=d != 0)


def compute_s(ru, mi):
    return np.sqrt(ru**2 + mi**2)
    # return np.where(np.isnan(ru), mi, np.sqrt(ru + np.nan_to_num(mi)))

def compute_metrics(pred, gt, toi, tau_arr, ic_arr=None, n_cpu=0):
    """
    Takes the prediction and the ground truth and for each threshold in tau_arr
    calculates the confusion matrix and returns the coverage,
    precision, recall, remaining uncertainty and misinformation.
    Toi is the list of terms (indexes) to be considered
    """
    g = gt[:, toi]
    n_gt = g.sum(axis=1)
    wn_gt = None
    if ic_arr is not None:
        #wn_gt = (g * ic_arr[toi]).sum(axis=1)
        wn_gt =np.dot(g,ic_arr[toi])

    
    
    
    arg_lists = [[tau_arr, g, pred, toi, n_gt, wn_gt, ic_arr] for tau_arr in np.array_split(tau_arr, 1)]
    #print(f" Nº  de cpus {n_cpu}, nº de args {len(   arg_lists)}")
    #with mp.Pool(processes=n_cpu) as pool:
    #    metrics = np.concatenate(pool.starmap(compute_metrics_, arg_lists), axis=0)

    results = []
    for args in arg_lists:
        result = compute_metrics_(*args)
        results.append(result)

    metrics = np.concatenate(results, axis=0)
    metrics=pd.DataFrame(metrics, columns=["cov", "pr", "rc", "wpr", "wrc", "ru", "mi"])
    for column in ["pr", "rc", "wpr", "wrc", "ru", "mi"]:
        metrics[column] = np.divide(metrics[column], metrics["cov"], out=np.zeros_like(metrics[column], dtype='float'), where=metrics["cov"] > 0)

    ne = np.full(len(tau_arr), gt.shape[0])
    metrics['ns'] = [""] * len(tau_arr)
    metrics['tau'] = tau_arr
    metrics['cov'] = np.divide(metrics['cov'], ne, out=np.zeros_like(metrics['cov'], dtype='float'), where=ne > 0)
    metrics['f'] = compute_f(metrics['pr'], metrics['rc'])
    metrics['wf'] = compute_f(metrics['wpr'], metrics['wrc'])
    metrics['s'] = compute_s(metrics['ru'], metrics['mi'])

    return metrics

# Return a mask for all the predictions (matrix) >= tau
def solidify_prediction(pred, tau):
    return pred >= tau
def custom_f1_score(y_validate,y_pred):
            max_scores={}
            max_umbrales={}
            scores_dict={}
            for aspect,indice in aspects.items():
                scores_dict[aspect]={}
                max_score=0
                max_umbral=0
                for umbral in np.arange(0.05,0.5,0.05):

                    y_pred_=(y_pred>umbral).astype(int)[:,indice]
                    scores_sums=y_pred_.sum(axis=1)
            
                    n=(scores_sums>0.5).sum()
                    #print(f"n = {n} Secuencias con algun score 1 {n/len(y_pred_)}")
                    y_validate_=y_validate.values[:,indice]

                    y_validate_sums=y_validate_.sum(axis=1)
                    w_use_=w_use[indice,:]

                    f1_scores = f1_score(y_validate_, y_pred_, average=None)
                    
                    t_p=y_pred_*y_validate_
                    
                    numerator=(np.dot(t_p,w_use_))
                    precision_denom=np.dot(y_pred_,w_use_)
                    precision_denom[precision_denom==0]=1
                    weighted_precision= np.sum(   numerator /  precision_denom) / n


                    
                    recall_denom= np.dot(y_validate_,w_use_)
                    recall_denom[recall_denom==0]=1

                
                    weighted_recall=    np.sum(numerator /   recall_denom) / len(y_pred_)
                    average_f1_score = 2*weighted_precision* weighted_recall/( weighted_recall+ weighted_precision)
            
                    scores_dict[aspect][umbral]= average_f1_score 
                # Calcula el F1-score promedio
                #average_f1_score = np.mean(f1_scores)
                    if  average_f1_score > max_score:
                        max_score=average_f1_score 
                        max_umbral=umbral
                max_scores[aspect]= max_score
                max_umbrales[aspect]=umbral
            
            max_score=np.mean(list(max_scores.values()))
            print(f"Scores {scores_dict}")
            # Print the F1 score
            print('F1 Score:', max_score)

class PredictionFnCallback(tf.keras.callbacks.Callback):
    
    def __init__(self,X_validate,y_validate, model=None, verbose=0):
        
               if not model is None: self.model = model
               self.verbose = verbose
               self.X_validate = X_validate
               self.y_validate = y_validate
               self.num_classes=num_classes
            
    def on_epoch_end(self, epoch, logs=None):
       
        y_pred=(self.model.predict(self.X_validate))
        #custom_f1_score(y_validate,y_pred)
        ms = []
        w_use=weights.loc[self.y_validate.columns].values
        ms_=(compute_metrics(y_pred, self.y_validate.values, list(range(self.y_validate.shape[1])), np.arange(0.1,0.5,0.05), ic_arr=w_use, n_cpu=0))
        ms.append(ms_.wf.max())
        print(np.mean(ms))
            

def custom_f1_score(y_validate,y_pred):
        #custom_f1_score(y_validate,y_pred)
        ms = []
        w_use=weights.loc[y_validate.columns].values
        ms_=(compute_metrics(y_pred,y_validate.values, list(range(y_validate.shape[1])), np.arange(0.1,0.5,0.05), ic_arr=w_use, n_cpu=0))
        ms.append(ms_.wf.max())
        print(np.mean(ms))



In [39]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
models = []
EPOCH_LOAD = "12"
VALIDATE =True
TRAIN =False
NUM_LABELS_dict= {        "BPO": 2000,
                              "CCO":2000,
                              "MFO": 1000}
ys={
        "CCO" : pd.read_pickle(os.path.join(PATH_LABELS,f"labels_{   NUM_LABELS_dict['CCO']}_CCO.pkl")),
        "MFO" : pd.read_pickle(os.path.join(PATH_LABELS,f"labels_{   NUM_LABELS_dict['MFO']}_MFO.pkl")),
        "BPO" : pd.read_pickle(os.path.join(PATH_LABELS,f"labels_{   NUM_LABELS_dict['BPO']}_BPO.pkl")),
    }
if TRAIN:
    
   
    for fold,(train_indice, test_indice) in enumerate(kfold.split(range(len(X)))):
        for aspect in [  "BPO", "MFO","CCO",]:
        

            y= ys[aspect]

            #X=np.concatenate([X_train,X_validate])
            #y=pd.concat([y_train,y_validate])

            num_classes=y.shape[1]
            print(f" Num classes {num_classes}")

            # model = CNN1D( num_classes)
            model =MultiLayerPerceptron(  num_classes)
            # Define the optimizer
            #optimizer=tf.keras.optimizers.Adam(learning_rate=CustomSchedule(LEARNING_RATE, WARMUP_STEPS), beta_1=0.9, beta_2=0.98, epsilon=1e-9)
            optimizer = tf.keras.optimizers.Adam()

            # Define the loss function
            loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
            model.compile(optimizer=optimizer,loss=loss_fn,metrics=['binary_accuracy', tf.keras.metrics.AUC()])


            
            checkpoint_path = PATHS["CHECKPOINT"][config.env].format(aspect, fold) + '/modelo-{epoch:02d}'
            checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_path,
                save_weights_only=False,
                save_freq='epoch',

            )



            model.fit(x=X[train_indice],y=y.iloc[train_indice],epochs=12,batch_size=32, callbacks=[checkpoint_callback,PredictionFnCallback(X[test_indice],y.iloc[test_indice])],)
            #models.append(model)
            del model


        if VALIDATE:
            print("Validating ...")
            predictions_t=None
  
            for aspect in [  "BPO", "MFO","CCO",]:
                y= ys[aspect]

                loaded_model = tf.keras.models.load_model( PATHS["CHECKPOINT"][config.env].format(aspect,fold)+f'/modelo-{EPOCH_LOAD}')
                
                # Use the loaded model for predictions or further operations
                predictions = loaded_model.predict(X[test_indice])

                custom_f1_score(y.iloc[test_indice],np.round(predictions,3) )

                print(np.round(predictions,3)[:10])
                predictions=np.round(predictions,3)
                predictions=pd.DataFrame(predictions,index=y.iloc[test_indice].index,columns=y.iloc[test_indice].columns,)
                predictions=predictions.stack()
                predictions=predictions[predictions>0]
                print(predictions.shape)
                predictions_t=pd.concat([predictions_t,predictions])
            predictions_t=predictions_t.sort_values(ascending=False).groupby(level=0).head(1500)

            predictions_t.to_csv(PATHS["PREDICTIONS"][config.env].format(fold),header=False, index=True, sep="\t")
            labels.loc[labels.EntryID.isin(y.iloc[test_indice].index)].to_csv(PATHS["LABELS"][config.env].format(fold),sep="\t",index=False)

In [40]:
def get_dataset( datatype, embeddings_source):
        if embeddings_source in ["ProtBERT", "EMS2"]:
            embeds = np.load(f"{PATH}" + embeds_map[embeddings_source] + "/" + datatype + "_embeddings.npy")
            ids = np.load(f"{PATH}" + embeds_map[embeddings_source] + "/" + datatype + "_ids.npy")
        
        if embeddings_source == "T5":
            embeds = np.load(f"{PATH}"  + embeds_map[embeddings_source] + "/" + datatype + "_embeds.npy")
            ids = np.load(f"{PATH}"  + embeds_map[embeddings_source] + "/" + datatype + "_ids.npy")
            

        if datatype=="train":
            df_labels=pd.read_pickle(PATH_LABELS)
            
            return embeds,ids,df_labels.loc[ids]
        return embeds,ids
        
 

In [41]:
EPOCH_LOAD = "12"

import gc 


X_test, ids_test= get_dataset( "test", embeddings_source)

predictions= None
preds= []
for aspect in [ "CCO", "MFO","BPO",]:
    
    y= ys[aspect]
    predictions_test_df_t =None
    for fold in range(n_splits):
        if TRAIN:
            loaded_model = tf.keras.models.load_model( PATHS["CHECKPOINT"][config.env].format(aspect,fold)+f'/modelo-{EPOCH_LOAD}')
        else:
            loaded_model = tf.keras.models.load_model( f'../results/preds_bp_aspect_kaggle/models_{aspect}_{fold}/modelo-{EPOCH_LOAD}')
        # Use the loaded model for predictions or further operations

        predictions_test = np.round(loaded_model.predict(X_test),3)
        del loaded_model
        gc.collect()
        print(predictions_test.shape)
        print(predictions_test[:10])


        print(len(ids_test),len(y.columns))
        predictions_test=pd.DataFrame(predictions_test,index=ids_test,columns=y.columns,)
        print((predictions_test==0).sum().max())
        
        
        

        predictions_test=predictions_test.stack()
        #predictions_test=predictions_test[predictions_test>0]

        if predictions_test_df_t is None:
            predictions_test_df_t = predictions_test
        else:
            predictions_test_df_t = predictions_test_df_t+ predictions_test
        del predictions_test
        gc .collect()
                
    predictions_test_df_t=predictions_test_df_t/n_splits
    preds.append(predictions_test_df_t)
predictions=pd.concat(preds)
    
        

                

#predictions= predictions.sort_values(ascending=False).groupby(level=0).head(1500)    


        
        

            

print(predictions.shape)

predictions=predictions [predictions >0]
print(predictions.shape)

predictions.to_csv("submission.tsv",header=False, index=True, sep="\t")


df_submission = pd.read_csv("submission.tsv",header=None, sep="\t")
display(df_submission)
                            

4434/4434 [==============================] - 8s 2ms/step
(141865, 2000)
[[0.535 0.487 0.352 ... 0.    0.    0.   ]
 [0.669 0.587 0.486 ... 0.    0.    0.   ]
 [0.691 0.606 0.413 ... 0.    0.    0.   ]
 ...
 [0.534 0.576 0.389 ... 0.    0.    0.   ]
 [0.654 0.659 0.606 ... 0.    0.    0.   ]
 [0.364 0.393 0.258 ... 0.    0.    0.   ]]
141865 2000
141865
4434/4434 [==============================] - 8s 2ms/step
(141865, 2000)
[[0.679 0.683 0.542 ... 0.    0.    0.   ]
 [0.733 0.761 0.645 ... 0.    0.    0.   ]
 [0.804 0.764 0.653 ... 0.    0.    0.   ]
 ...
 [0.622 0.541 0.433 ... 0.    0.    0.   ]
 [0.795 0.745 0.642 ... 0.    0.    0.   ]
 [0.431 0.491 0.414 ... 0.    0.    0.   ]]
141865 2000
141865
4434/4434 [==============================] - 7s 2ms/step
(141865, 2000)
[[0.614 0.585 0.323 ... 0.    0.    0.   ]
 [0.675 0.666 0.387 ... 0.    0.    0.   ]
 [0.717 0.705 0.403 ... 0.    0.    0.   ]
 ...
 [0.707 0.642 0.424 ... 0.    0.    0.   ]
 [0.84  0.803 0.707 ... 0.    0.    0.   

,0,1,2
0,Q9CQV8,GO:0110165,0.6192
1,Q9CQV8,GO:0005622,0.5508
2,Q9CQV8,GO:0043226,0.3994
3,Q9CQV8,GO:0043229,0.3684
4,Q9CQV8,GO:0043227,0.3144
...,...,...,...
266227808,A0A3G2FQK2,GO:0031102,0.0004
266227809,A0A3G2FQK2,GO:0042048,0.0002
266227810,A0A3G2FQK2,GO:0031670,0.0002
266227811,A0A3G2FQK2,GO:0002821,0.0002


In [42]:
import pandas as pd

df_submission = pd.read_csv("submission.tsv",header=None, sep="\t")

In [43]:
df_submission .groupby(0).size().max()

3791

In [44]:
tf.__version__

'2.13.0'